In [2]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [34]:
# Tensorflow 1.15버전으로
# Gate 연산을 수행하는 Logistic Regression으로 구현해보아요

import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report


# Training Data set
x_data = np.array([[0,0],
                  [0,1],
                  [1,0],
                  [1,1]], dtype = np.float32)
t_data = np.array([[0],[1],[1],[0]], dtype = np.float32)



# placeholder 

X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# Weight & bias

W = tf.Variable(tf.random.normal([2,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# Hypothesis

logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,
                                                             labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# session, 초기화
sess  = tf.Session()
sess.run(tf.global_variables_initializer())


# 학습
for step in range(30000):
    _, loss_val = sess.run([train, loss], feed_dict={X:x_data,
                                                    T:t_data})
    if step % 3000 == 0:
        print('loss : {}'.format(loss_val))
        
# 성능평가 ( Accuracy )
accuracy= tf.cast(H >= 0.5, dtype = tf.float32)
result = sess.run(accuracy, feed_dict={X:x_data})
print(classification_report(t_data.ravel(), result.ravel()))

loss : 0.880738377571106
loss : 0.693333089351654
loss : 0.6931636333465576
loss : 0.6931487321853638
loss : 0.6931473016738892
loss : 0.6931471824645996
loss : 0.6931471824645996
loss : 0.6931471824645996
loss : 0.6931471824645996
loss : 0.6931471824645996
              precision    recall  f1-score   support

         0.0       0.33      0.50      0.40         2
         1.0       0.00      0.00      0.00         2

    accuracy                           0.25         4
   macro avg       0.17      0.25      0.20         4
weighted avg       0.17      0.25      0.20         4



In [35]:
%reset

# Gate 연산으로 수행하는 Deep Learning으로 구현해보아요
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report


# Training Data set
x_data = np.array([[0,0],
                  [0,1],
                  [1,0],
                  [1,1]], dtype = np.float32)
t_data = np.array([[0],[1],[1],[0]], dtype = np.float32)



# placeholder 

X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# Weight & bias

W2 = tf.Variable(tf.random.normal([2,100]), name='weight2') 
# 몇개의 아웃풋이있나요?
# 로지스틱이 몇개있나요? 
b2 = tf.Variable(tf.random.normal([100]), name='bias2')
# 로지스틱이 100개있으니 
# bias도 100개
# shape만 변경해주면 됨 . 행렬 연산의 장점이다.
layer2 = tf.sigmoid(tf.matmul(X, W2) + b2) 


W3 = tf.Variable(tf.random.normal([100,6]), name='weight3') 
b3 = tf.Variable(tf.random.normal([6]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
# 지금까지는 히든 레이어는 2개



# 최종 layer
W4 = tf.Variable(tf.random.normal([6,1]), name='weight4') 
b4 = tf.Variable(tf.random.normal([1]), name='bias4')



# Hypothesis

logit = tf.matmul(layer3,W4) + b4
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,
                                                             labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# session, 초기화
sess  = tf.Session()
sess.run(tf.global_variables_initializer())


# 학습
for step in range(30000):
    _, loss_val = sess.run([train, loss], feed_dict={X:x_data,
                                                    T:t_data})
    if step % 3000 == 0:
        print('loss : {}'.format(loss_val))
        
# 성능평가 ( Accuracy )
accuracy= tf.cast(H >= 0.5, dtype = tf.float32)
result = sess.run(accuracy, feed_dict={X:x_data})
print(classification_report(t_data.ravel(), result.ravel()))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
loss : 1.1478232145309448
loss : 0.6937179565429688
loss : 0.6775369048118591
loss : 0.5920534133911133
loss : 0.5317761898040771
loss : 0.4170067608356476
loss : 0.20501229166984558
loss : 0.1168903112411499
loss : 0.08102734386920929
loss : 0.0620705671608448
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         2
         1.0       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [31]:
%reset


############################################


# MNIST -> multinomial classification
# 1.15버전 방식으로 단일 layer 구현

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler # Normalizeration
from sklearn.model_selection import train_test_split # train. test 데이터 분리
from sklearn.model_selection import KFold # cross validation

df = pd.read_csv('/Users/mac/notebook_dir/data/digit-recognizer/numtrain.csv')

x_data= df.drop('label',axis=1,inplace=False).values
t_data= df['label'].values

x_data_train, x_data_test,t_data_train, t_data_test=\
train_test_split(x_data,t_data,test_size=0.3,random_state=0)

# # Normalization 진행(MIN-MAX scaling)
# scaler=MinMaxScaler()
# scaler.fit(x_data_train) # 나중에 scaling을 하기 위한 정보를 scaler에게 세팅
# x_data_train_norm=scaler.transform(x_data_train) # scaling 진행
# x_data_test_norm=scaler.transform(x_data_test)   # scaling 진행

# del x_data_test  # 에러방지용으로 삭제
# del x_data_train # 에러방지용으로 삭제

sess= tf.Session()
# 우리가 사용할 label(t_data)를 one hot encoding 형태로 변환!!
t_data_train_onehot=sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot=sess.run(tf.one_hot(t_data_test, depth=10))

del t_data_train
del t_data_test


Once deleted, variables cannot be recovered. Proceed (y/[n])? y
